In [18]:
import pandas as pd, numpy as np
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error, r2_score

In [19]:
df = pd.read_csv('CLN_FLR_BGO_21_24.csv', delimiter=';')

In [20]:
df.head()

,Date,Arr,Dep,Flight In,Flight Out,Origin,Destination,STA,ATA,STD,ATD,Delay Code / Time,Aircraft Type,Aircraft Reg
0,01/01/2021,0,1,WF534,WF672,BGO,TRD,NaN,NaN,10:20,10:18,NaN,DH4,LNWDH
1,01/01/2021,1,0,WF586,NaN,KRS,BGO,10:25,10:13,NaN,NaN,NaN,DH4,LNWDG
2,01/01/2021,0,1,WF586,WF533,BGO,SVG,NaN,NaN,10:55,10:59,TD/00:04,DH4,LNWDG
3,01/01/2021,0,1,WF9160,WF568,BGO,KSU,NaN,NaN,11:00,10:57,NaN,DH4,LNWDJ
4,01/01/2021,0,1,WF623,WF614,BGO,BOO,NaN,NaN,12:55,12:55,NaN,290,LNWEB


In [21]:
dep_only_df = df[df['Dep'] == 1]
# Dropping columns that are not needed.
columns_to_drop = ['Arr', 'Flight In', 'STA', 'ATA',]
cleaned_df = dep_only_df.drop(columns=columns_to_drop)

In [22]:
cleaned_df.head()

,Date,Dep,Flight Out,Origin,Destination,STD,ATD,Delay Code / Time,Aircraft Type,Aircraft Reg
0,01/01/2021,1,WF672,BGO,TRD,10:20,10:18,NaN,DH4,LNWDH
2,01/01/2021,1,WF533,BGO,SVG,10:55,10:59,TD/00:04,DH4,LNWDG
3,01/01/2021,1,WF568,BGO,KSU,11:00,10:57,NaN,DH4,LNWDJ
4,01/01/2021,1,WF614,BGO,BOO,12:55,12:55,NaN,290,LNWEB
6,01/01/2021,1,WF106,BGO,FRO,13:25,13:30,RA/00:05,DH3,LNWFS


In [23]:
cleaned_df['Date'] = pd.to_datetime(cleaned_df['Date'], format='%d/%m/%Y')
cleaned_df['STD'] = pd.to_datetime(cleaned_df['STD'], format='%H:%M').dt.time
cleaned_df['ATD'] = pd.to_datetime(cleaned_df['ATD'], format='%H:%M').dt.time
cleaned_df['DayOfWeek'] = cleaned_df['Date'].dt.dayofweek
cleaned_df['Month'] = cleaned_df['Date'].dt.month
cleaned_df['STD_Minutes'] = cleaned_df['STD'].apply(lambda x: x.hour * 60 + x.minute)
cleaned_df['ATD_Minutes'] = cleaned_df['ATD'].apply(lambda x: x.hour * 60 + x.minute)
cleaned_df = cleaned_df.drop('Origin', axis=1)
cleaned_df = cleaned_df.drop('Delay Code / Time', axis=1)
cleaned_df['Flight Out'] = cleaned_df['Flight Out'].str.replace('WF','')
cleaned_df['Aircraft Type'] = cleaned_df['Aircraft Type'].str.replace('DH','')
cleaned_df['Aircraft Type'] = cleaned_df['Aircraft Type'].str.replace('AT','')
cleaned_df = cleaned_df.dropna()
# Verify the operation by checking for NaN values again
print(cleaned_df.isna().sum())

Date             0
Dep              0
Flight Out       0
Destination      0
STD              0
ATD              0
Aircraft Type    0
Aircraft Reg     0
DayOfWeek        0
Month            0
STD_Minutes      0
ATD_Minutes      0
dtype: int64


In [24]:
cleaned_df.head()

,Date,Dep,Flight Out,Destination,STD,ATD,Aircraft Type,Aircraft Reg,DayOfWeek,Month,STD_Minutes,ATD_Minutes
0,2021-01-01,1,672,TRD,10:20:00,10:18:00,4,LNWDH,4,1,620,618.0
2,2021-01-01,1,533,SVG,10:55:00,10:59:00,4,LNWDG,4,1,655,659.0
3,2021-01-01,1,568,KSU,11:00:00,10:57:00,4,LNWDJ,4,1,660,657.0
4,2021-01-01,1,614,BOO,12:55:00,12:55:00,290,LNWEB,4,1,775,775.0
6,2021-01-01,1,106,FRO,13:25:00,13:30:00,3,LNWFS,4,1,805,810.0


In [25]:
# For destination
le_destination = LabelEncoder()
cleaned_df['Destination'] = le_destination.fit_transform(cleaned_df['Destination'])

# Extracting the mapping
destination_mapping = dict(zip(le_destination.classes_, le_destination.transform(le_destination.classes_)))
destination_mapping_df = pd.DataFrame(list(destination_mapping.items()), columns=['Destination', 'Encoded_Value'])

# For aircraft reg
le_aircraft_reg = LabelEncoder()
cleaned_df['Aircraft Reg'] = le_aircraft_reg.fit_transform(cleaned_df['Aircraft Reg'])

# Extracting the mapping
aircraft_reg_mapping = dict(zip(le_aircraft_reg.classes_, le_aircraft_reg.transform(le_aircraft_reg.classes_)))
aircraft_reg_mapping_df = pd.DataFrame(list(aircraft_reg_mapping.items()), columns=['Aircraft Reg', 'Encoded_Value'])

print(destination_mapping_df)

   Destination  Encoded_Value
0          ABZ              0
1          ACE              1
2          AES              2
3          AGP              3
4          AJA              4
..         ...            ...
82         TRD             82
83         TRF             83
84         UME             84
85         VCE             85
86         ZRH             86

[87 rows x 2 columns]


In [31]:
from joblib import dump
dump(le_destination, 'airport_encoder.joblib')

['airport_encoder.joblib']

In [26]:
unique_aircraft_types = cleaned_df['Aircraft Type'].unique()
print(unique_aircraft_types)

['4' '290' '3' '1' '2' '7']


In [27]:
X = cleaned_df.drop(['ATD_Minutes', 'ATD','STD', 'Date','Dep','Aircraft Type',
                    'Aircraft Reg'], axis=1)
y = cleaned_df['ATD_Minutes']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [28]:
print(X.columns)

Index(['Flight Out', 'Destination', 'DayOfWeek', 'Month', 'STD_Minutes'], dtype='object')


In [29]:
rf = RandomForestRegressor(n_estimators=100, random_state = 42)
rf.fit(X_train, y_train)

predictions = rf.predict(X_test)

mae = mean_absolute_error(y_test, predictions)
mse = mean_squared_error(y_test, predictions)
rmse= mean_squared_error(y_test, predictions, squared=False)
r2 = r2_score(y_test, predictions)

print("Mean Squared Error:", mse)
print("Root mean squared error:", rmse)
print("R2-score", r2)
print("Mean Absolute Error:", mae)

Mean Squared Error: 2629.6346606519837
Root mean squared error: 51.27996353988548
R2-score 0.9669699924747039
Mean Absolute Error: 14.121987749882397


/Users/perry/Documents/flightetd/venv/lib/python3.9/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [30]:
import pickle

pickle.dump(rf, open("model.pkl", "wb"))

In [32]:
XP = cleaned_df[['Flight Out', 'Destination', 'DayOfWeek', 'Month', 'STD_Minutes']]

In [33]:
predictions = rf.predict(XP)

In [34]:
cleaned_df['predicted'] = predictions

In [35]:
print(cleaned_df.head())

        Date  Dep Flight Out  Destination       STD       ATD Aircraft Type  \
0 2021-01-01    1        672           82  10:20:00  10:18:00             4   
2 2021-01-01    1        533           77  10:55:00  10:59:00             4   
3 2021-01-01    1        568           42  11:00:00  10:57:00             4   
4 2021-01-01    1        614           13  12:55:00  12:55:00           290   
6 2021-01-01    1        106           28  13:25:00  13:30:00             3   

   Aircraft Reg  DayOfWeek  Month  STD_Minutes  ATD_Minutes   predicted  
0             7          4      1          620        618.0  620.845365  
2             6          4      1          655        659.0  654.447131  
3             9          4      1          660        657.0  653.127726  
4            18          4      1          775        775.0  773.925333  
6            22          4      1          805        810.0  799.223833  
